## Finding Low Accuracy Cluster

In [2]:
# LIBRARIES
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [1]:
# PARAMETERS
filepath = 'prepared_data/german.csv'

trial_cnt = 10

In [3]:
# GET DATA
df = pd.read_csv(filepath)

y_cols = ['y_true','y_pred_score','y_pred']
X_df =  df.loc[:, ~df.columns.isin(y_cols)]

X = X_df.to_numpy()
X_scaled = StandardScaler().fit_transform(X)

In [89]:
# We want to find a cluster with the lowest accuracy
 # - h

    
# Create several different clusters, then check the accuracy of all of them
def basicClustering(x,y_true,y_pred,k,trial_cnt):
    result_accuracy=[1]*k
    result_clusters=1
    
    for t in range(10):
        accuracy=[None]*k
        size=[None]*k

        clusters = KMeans(k).fit(X_scaled) 
        
        # calculate the accuracy for each of the clusters
        for i in np.unique(clusters.labels_):
            mask = (clusters.labels_ == i)

            # cluster_x = X_scaled[mask,:]
            cluster_y_true = y_true[mask]
            cluster_y_pred = y_pred[mask]
            

            accuracy[i] = accuracy_score(cluster_y_true,cluster_y_pred)
            size[i] = cluster_y_true.shape[0]

        k_min = accuracy.index(min(accuracy))
        # check if the cluster with the smallest accuracy is the min so far
        if min(accuracy)<min(result_accuracy):
            result_accuracy=accuracy
            result_clusters = clusters

        print('trial '+str(t)+': '+str(accuracy[k_min])+' ('+str(size[k_min])+')')
    return(result_accuracy,result_clusters)


In [81]:
# sub cluster

def getLeastAccurateCluster(x,y_true,y_pred,k,n_min):
    accuracy=[None]*k
    size=[None]*k
    clusters = KMeans(k).fit(x)
    for i in np.unique(clusters.labels_):
        mask = (clusters.labels_ == i)
        if sum(mask) >= n_min:
            cluster_x = x[mask,:]
            cluster_y_true = y_true[mask]
            cluster_y_pred = y_pred[mask]

            accuracy[i] = accuracy_score(cluster_y_true,cluster_y_pred)
        else:
            accuracy[i] = 1
        size[i]=sum(mask)
    k_min = accuracy.index(min(accuracy))
    mask = clusters.labels_ == k_min
    center = clusters.cluster_centers_[k_min,:]

    return(mask,min(accuracy),center)


def getLeastAccurateCluster_Recursive(x,y_true,y_pred,k,n_min):
    # print('TOP x shape:',x.shape)
    n = x.shape[0]
    indices = list(range(0, n))
    first_mask,first_accuracy,first_center = getLeastAccurateCluster(x,y_true,y_pred,k,n_min)

    return_mask = first_mask
    return_accuracy = first_accuracy
    return_center = first_center
    
    if sum(first_mask) > n_min:
        # print('first_mask shape:',first_mask.shape)
        # print('y_true shape:',y_true.shape)
        new_x = x[first_mask,:]
        new_y_true = y_true[first_mask]
        new_y_pred = y_pred[first_mask]
        
        full_i = np.array(list(range(0, n)))
        sub_i = full_i[first_mask]
        
        sub_mask,sub_accuracy,sub_center = getLeastAccurateCluster_Recursive(new_x,new_y_true,new_y_pred,k,n_min)
        
        if sub_accuracy < return_accuracy:
            return_mask = np.full(n,False)
            for i in sub_i[sub_mask]:
                return_mask[i] = True
            return_accuracy = sub_accuracy
            return_center = sub_center
        
        # return(return_mask)
    
    # else: return(newMask)
    return(return_mask,return_accuracy,return_center)


def subClustering(x,y_true,y_pred,k,n_min,trial_cnt):
    result_accuracy=1
    result_center=1
    result_mask = []
    
    for t in range(10):
        accuracy=[None]*k
        # size=[None]*k
        
        mask,accuracy,center = getLeastAccurateCluster_Recursive(x,y_true,y_pred,k,n_min)
        # print(mask)

        # check if the cluster with the smallest accuracy is the min so far
        if accuracy<result_accuracy:
            result_accuracy=accuracy
            result_center = center
            result_mask = mask

    # print(resul_mask)
    # print(mask)
        print('trial '+str(t)+': '+str(accuracy)+' ('+str(sum(1*mask))+')')
    return(result_accuracy,result_center,result_mask)




In [97]:
k = 5
trial_cnt = 10

y_true = df['y_true']
y_pred = df['y_pred']


clust_accuracy, clusters = basicClustering(X_scaled,y_true,y_pred,k,trial_cnt)

k_min = clust_accuracy.index(min(clust_accuracy))
# clusters.cluster_centers_[k_min,:]
# k_min

trial 0: 0.7028985507246377 (138)
trial 1: 0.6896551724137931 (29)
trial 2: 0.7094017094017094 (117)
trial 3: 0.6666666666666666 (63)
trial 4: 0.6458333333333334 (48)
trial 5: 0.6896551724137931 (29)
trial 6: 0.7101449275362319 (138)
trial 7: 0.7049180327868853 (61)
trial 8: 0.68 (50)
trial 9: 0.64 (50)


In [96]:
k = 5
trial_cnt = 10
n_min = 10

y_true = df['y_true']
y_pred = df['y_pred']


accuracy,center,mask = subClustering(X_scaled,y_true,y_pred,k,n_min,trial_cnt)

np.mean(accuracy)


trial 0: 0.6875 (32)
trial 1: 0.6896551724137931 (29)
trial 2: 0.5 (14)
trial 3: 0.6 (10)
trial 4: 0.5 (12)
trial 5: 0.6896551724137931 (29)
trial 6: 0.5925925925925926 (27)
trial 7: 0.6896551724137931 (29)
trial 8: 0.6 (10)
trial 9: 0.5714285714285714 (21)


0.5

In [23]:
result_clusters.cluster_centers_.shape
c = result_clusters.cluster_centers_[k_min,:]

coef_df=pd.DataFrame()
coef_df['i'] = np.absolute(c).argsort()[::-1]
coef_df['col'] = X_df.columns[coef_df['i']]
coef_df['val'] = c[coef_df['i']]
coef_df

,i,col,val
0,14,credit_history=A32,0.769891
1,5,number_of_credits,-0.613972
2,16,credit_history=A34,-0.591402
3,33,employment=A72,0.341182
4,37,other_debtors=A101,0.327693
5,49,housing=A153,-0.321992
6,43,property=A124,-0.309800
7,4,age,-0.300702
8,15,credit_history=A33,-0.291021
9,39,other_debtors=A103,-0.254000
